In [2]:
import numpy as np
import pandas as pd
import pydicom
%matplotlib inline
import matplotlib.pyplot as plt
import keras 

from skimage.transform import resize
from keras.models import model_from_json

In [20]:
# This function reads in a .dcm file, checks the important fields for our device, and returns a numpy array
# of just the imaging data
def check_dicom(filename): 
    print('Load file {} ...'.format(filename))
    ds = pydicom.dcmread(filename)       
    img = ds.pixel_array
    img_mean = img.mean()
    img_std = img.std()
    img_type = ds.Modality
    
    img_bodyp=ds.BodyPartExamined
    img_position=ds.PatientPosition
    
    if  img_type != 'DX' or \
        img_bodyp != 'CHEST' or \
        (img_position != 'AP' and img_position != 'PA'):
        print('Unsupported dycom file {} {} {}\n'.format(img_type, img_bodyp, img_position))
        return None
    else:
        return img,img_mean,img_std
    
    
# This function takes the numpy array output by check_dicom and 
# runs the appropriate pre-processing needed for our model input
def preprocess_image(img,img_mean,img_std,img_size): 
    img = (img - img_mean) / img_std
    proc_img = resize(img, img_size)
    return proc_img

# This function loads in our trained model w/ weights and compiles it 
def load_model(model_path, weight_path):
    with open(model_path, 'r') as json_file:
        loaded_model_json = json_file.read()
    
        model = model_from_json(loaded_model_json)
    
        # load weights into new model
        model.load_weights(weight_path)

        # evaluate loaded model on test data
        model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['binary_accuracy'])
    
    return model

# This function uses our device's threshold parameters to predict whether or not
# the image shows the presence of pneumonia using our trained model
def predict_image(model, img, thresh): 
    prediction = model.predict(img)
    return 'Pneumonia' if prediction > thresh else 'No pneumonia'

In [22]:
test_dicoms = ['test1.dcm','test2.dcm','test3.dcm','test4.dcm','test5.dcm','test6.dcm']

model_path = 'my_model.json' #path to saved model
weight_path = 'xray_class_my_model.best.hdf5' #path to saved best weights

IMG_SIZE=(1,224,224,3) # This might be different if you did not use vgg16
# img_mean = # loads the mean image value they used during training preprocessing
# img_std = # loads the std dev image value they used during training preprocessing

my_model = load_model(model_path, weight_path)
thresh = 0.4#loads the threshold they chose for model classification 

# use the .dcm files to test your prediction
for i in test_dicoms:
    
    img = np.array([])
    if check_dicom(i) == None:
        continue
        
    img,img_mean,img_std = check_dicom(i)
        
    img_proc = preprocess_image(img,img_mean,img_std,IMG_SIZE)
    pred = predict_image(my_model,img_proc,thresh)
    print(pred + '\n')

Load file test1.dcm ...
Load file test1.dcm ...
Pneumonia

Load file test2.dcm ...
Load file test2.dcm ...
Pneumonia

Load file test3.dcm ...
Load file test3.dcm ...
Pneumonia

Load file test4.dcm ...
Unsupported dycom file DX RIBCAGE PA

Load file test5.dcm ...
Unsupported dycom file CT CHEST PA

Load file test6.dcm ...
Unsupported dycom file DX CHEST XX

